## Airline Tweets pre-processing

In [30]:
# Import các thư viện 
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

import nltk
from bs4 import BeautifulSoup
from nltk.stem import WordNetLemmatizer

import re
import glob
import os
import pandas as pd
from nltk.corpus import stopwords
from wordcloud import WordCloud, STOPWORDS
import spacy
from spacy.lang.en import English
from nltk import word_tokenize, pos_tag, ne_chunk
from autocorrect import Speller
from nltk.stem import PorterStemmer
from textblob import TextBlob

In [31]:
# Đọc dữ liệu từ tệp CSV vào DataFrame
df = pd.read_csv('../data/final_train_df.csv', encoding="ISO-8859-1")

# Trộn ngẫu nhiên các dòng trong DataFrame
df_shuffled=df.sample(frac=1).reset_index(drop=True)

df.head()

,tweet_id,airline_sentiment,airline_sentiment_confidence,negativereason,negativereason_confidence,airline,airline_sentiment_gold,name,negativereason_gold,retweet_count,text,tweet_coord,tweet_created,tweet_location,user_timezone
0,570306133677760513,neutral,1.0000,NaN,NaN,Virgin America,NaN,cairdin,NaN,0,@VirginAmerica What @dhepburn said.,NaN,2015-02-24 11:35:52 -0800,NaN,Eastern Time (US & Canada)
1,570301130888122368,positive,0.3486,NaN,0.0000,Virgin America,NaN,jnardino,NaN,0,@VirginAmerica plus you've added commercials t...,NaN,2015-02-24 11:15:59 -0800,NaN,Pacific Time (US & Canada)
2,570301083672813571,neutral,0.6837,NaN,NaN,Virgin America,NaN,yvonnalynn,NaN,0,@VirginAmerica I didn't today... Must mean I n...,NaN,2015-02-24 11:15:48 -0800,Lets Play,Central Time (US & Canada)
3,570301031407624196,negative,1.0000,Bad Flight,0.7033,Virgin America,NaN,jnardino,NaN,0,@VirginAmerica it's really aggressive to blast...,NaN,2015-02-24 11:15:36 -0800,NaN,Pacific Time (US & Canada)
4,570300817074462722,negative,1.0000,Can't Tell,1.0000,Virgin America,NaN,jnardino,NaN,0,@VirginAmerica and it's a really big bad thing...,NaN,2015-02-24 11:14:45 -0800,NaN,Pacific Time (US & Canada)


In [32]:
# Phân chia dữ liệu theo cảm xúc
positive_df = df[df['airline_sentiment'] == 'positive']  # Lọc các tweet có cảm xúc tích cực
neutral_df = df[df['airline_sentiment'] == 'neutral']  # Lọc các tweet có cảm xúc trung lập
negative_df = df[df['airline_sentiment'] == 'negative']  # Lọc các tweet có cảm xúc tiêu cực

### Initial pre-processing

In [33]:
# Loại bỏ các dòng có dữ liệu trùng lặp trong cột 'text'
df = df.drop_duplicates(['text'])

# Kiểm tra và xóa các cột không cần thiết
columns_to_drop = ['Unnamed: 0', 'tweet_id']  # Các cột cần loại bỏ
df = df.drop(columns=[col for col in columns_to_drop if col in df.columns], axis=1)  # Loại bỏ các cột

df.head()


,airline_sentiment,airline_sentiment_confidence,negativereason,negativereason_confidence,airline,airline_sentiment_gold,name,negativereason_gold,retweet_count,text,tweet_coord,tweet_created,tweet_location,user_timezone
0,neutral,1.0000,NaN,NaN,Virgin America,NaN,cairdin,NaN,0,@VirginAmerica What @dhepburn said.,NaN,2015-02-24 11:35:52 -0800,NaN,Eastern Time (US & Canada)
1,positive,0.3486,NaN,0.0000,Virgin America,NaN,jnardino,NaN,0,@VirginAmerica plus you've added commercials t...,NaN,2015-02-24 11:15:59 -0800,NaN,Pacific Time (US & Canada)
2,neutral,0.6837,NaN,NaN,Virgin America,NaN,yvonnalynn,NaN,0,@VirginAmerica I didn't today... Must mean I n...,NaN,2015-02-24 11:15:48 -0800,Lets Play,Central Time (US & Canada)
3,negative,1.0000,Bad Flight,0.7033,Virgin America,NaN,jnardino,NaN,0,@VirginAmerica it's really aggressive to blast...,NaN,2015-02-24 11:15:36 -0800,NaN,Pacific Time (US & Canada)
4,negative,1.0000,Can't Tell,1.0000,Virgin America,NaN,jnardino,NaN,0,@VirginAmerica and it's a really big bad thing...,NaN,2015-02-24 11:14:45 -0800,NaN,Pacific Time (US & Canada)


### Removing special characters

In [34]:
# Hàm làm sạch văn bản
def clean(txt):
    txt = txt.replace("()", "")  # Loại bỏ các ký tự "()"
    txt = txt.replace('(<a).*(>).*()', '')  # Loại bỏ các thẻ HTML
    txt = txt.replace('(&amp)', '')  # Loại bỏ các ký tự đặc biệt như &amp
    txt = txt.replace('(&gt)', '')  # Loại bỏ các ký tự đặc biệt như &gt
    txt = txt.replace('(&lt)', '')  # Loại bỏ các ký tự đặc biệt như &lt
    txt = txt.replace('(\xa0)', ' ')  # Loại bỏ ký tự không phải không gian
    return txt

# Áp dụng hàm làm sạch cho cột 'text' trong DataFrame
df['text'] = df['text'].apply(lambda x: clean(x))
df['text'].head()

0                  @VirginAmerica What @dhepburn said.
1    @VirginAmerica plus you've added commercials t...
2    @VirginAmerica I didn't today... Must mean I n...
3    @VirginAmerica it's really aggressive to blast...
4    @VirginAmerica and it's a really big bad thing...
Name: text, dtype: object

### Extracting all the hastags

In [35]:
# Trích xuất các hashtag và mentions từ văn bản
df['tweet_hastags'] = df['text'].apply(lambda x: re.findall("#([a-zA-Z0-9_]{1,50})", x))  # Tìm tất cả các hashtag
df['tweet_hastags'].value_counts()  # Đếm số lần xuất hiện của các hashtag

[]                      12077
[DestinationDragons]       70
[fail]                     36
[usairwaysfail]            21
[customerservice]          21
                        ...  
[peanutsonaplatter]         1
[letitgo]                   1
[hotlanta]                  1
[notmadeofmoney]            1
[BlackBerry10]              1
Name: tweet_hastags, Length: 1785, dtype: int64

### Extracting all the mentions

In [36]:
df['mentions'] = df['text'].apply(lambda x: re.findall("@([a-zA-Z0-9_]{1,50})", x))  # Tìm tất cả các mentions
df['mentions'].value_counts()  # Đếm số lần xuất hiện của các mentions

[united]                          3370
[USAirways]                       2470
[AmericanAir]                     2281
[SouthwestAir]                    2090
[JetBlue]                         1932
                                  ... 
[SouthwestAir, TifffyHuang]          1
[SouthwestAir, JasonWhitely]         1
[SouthwestAir, SwagglikeBean]        1
[SouthwestAir, SacIntlAirport]       1
[AmericanAir, TilleyMonsta]          1
Name: mentions, Length: 1020, dtype: int64

### Removing hastags and mentions from tweets 

In [37]:
# Làm sạch annotation (xóa mentions và hashtags)
def clean_annotation(tweet):
    clean_tweet = re.sub("@[A-Za-z0-9_]+", "", tweet)  # Xóa mentions
    clean_tweet = re.sub("#[A-Za-z0-9_]+", "", clean_tweet)  # Xóa hashtags
    return clean_tweet

# Áp dụng hàm làm sạch annotation cho cột 'text'
df['tweets'] = df['text'].apply(lambda x: clean_annotation(x))
df['tweets'].head()


0                                          What  said.
1     plus you've added commercials to the experien...
2     I didn't today... Must mean I need to take an...
3     it's really aggressive to blast obnoxious "en...
4             and it's a really big bad thing about it
Name: tweets, dtype: object

### Removing HTTP Links 

In [38]:
#Loại bỏ URL và ký tự đặc biệt
df['tweets'] = df['tweets'].apply(lambda x: re.sub(r'https?:\/\/\S*', '', x, flags=re.MULTILINE))
df['tweets'].head()


0                                          What  said.
1     plus you've added commercials to the experien...
2     I didn't today... Must mean I need to take an...
3     it's really aggressive to blast obnoxious "en...
4             and it's a really big bad thing about it
Name: tweets, dtype: object

### Converting to Lowercase

In [39]:
# Chuyển đổi tất cả các từ về chữ thường
df['tweets'] = df['tweets'].apply(lambda x: " ".join(x.lower() for x in x.split()))
df['tweets'].head()


0                                           what said.
1    plus you've added commercials to the experienc...
2    i didn't today... must mean i need to take ano...
3    it's really aggressive to blast obnoxious "ent...
4             and it's a really big bad thing about it
Name: tweets, dtype: object

### Removing punctuation

In [40]:
# Loại bỏ ký tự đặc biệt không phải từ hoặc khoảng trắng
df['tweets'] = df['tweets'].str.replace('[^\w\s]','')
df['tweets'].head()

0                                            what said
1    plus youve added commercials to the experience...
2    i didnt today must mean i need to take another...
3    its really aggressive to blast obnoxious enter...
4              and its a really big bad thing about it
Name: tweets, dtype: object

### De-emojify tweets to sentiments

In [41]:
# Xóa emoji trong cột 'tweets'
f = open("../Notebooks/emoji_regex.txt", "r")
def remove_emoji(text):
    emoji_pattern = re.compile(f.read(), flags=re.UNICODE)
    return emoji_pattern.sub(r'', text)

In [42]:
df['tweets'] = df['tweets'].apply(lambda x: remove_emoji(x))  # Áp dụng loại bỏ emoji
df['tweets'].head()

0                                            what said
1    plus youve added commercials to the experience...
2    i didnt today must mean i need to take another...
3    its really aggressive to blast obnoxious enter...
4              and its a really big bad thing about it
Name: tweets, dtype: object

### Lemmatization

In [43]:
# Chuyển đổi các từ về dạng gốc (lemmatization) sử dụng spacy
import spacy
load_model = spacy.load('en_core_web_sm', disable=['parser', 'ner'])  # Tải mô hình ngôn ngữ Anh nhỏ của spacy

In [44]:
def lemmatize(x):
    doc = load_model(x)  # Tạo đối tượng Doc từ văn bản
    return " ".join([token.lemma_ for token in doc])  # Lấy dạng gốc của từ

In [45]:
df['tweets'] = df['tweets'].apply(lambda x: lemmatize(x))  # Áp dụng lemmatization cho cột 'tweets'
df['tweets'].head()

0                                             what say
1    plus you ve add commercial to the experience t...
2    I do not today must mean I need to take anothe...
3    its really aggressive to blast obnoxious enter...
4              and its a really big bad thing about it
Name: tweets, dtype: object

### Part of speech tagging (POS)

In [46]:
# Phân tích cú pháp POS (Part of Speech) cho mỗi từ trong tweet
from textblob import TextBlob

def pos_tag(x):
    result = TextBlob(x)  # Chuyển đổi văn bản thành đối tượng TextBlob
    return result.tags  # Trả về các nhãn POS (từ loại)

In [47]:
from textblob import TextBlob

df['tweets_tags'] = df['tweets'].apply(lambda x: TextBlob(x).tags)
df['tweets_tags'].head()


0                             [(what, WP), (say, VBP)]
1    [(plus, CC), (you, PRP), (ve, VBP), (add, VB),...
2    [(I, PRP), (do, VBP), (not, RB), (today, NN), ...
3    [(its, PRP$), (really, RB), (aggressive, JJ), ...
4    [(and, CC), (its, PRP$), (a, DT), (really, RB)...
Name: tweets_tags, dtype: object

In [48]:
from textblob import TextBlob

blob = TextBlob("TextBlob is working correctly now.")
print(blob.tags)


[('TextBlob', 'NNP'), ('is', 'VBZ'), ('working', 'VBG'), ('correctly', 'RB'), ('now', 'RB')]


### Named Entity recognition

In [61]:
# Định nghĩa hàm NER (Named Entity Recognition) và thực hiện stemming trên tweet
def ner(x):
    return ne_chunk(pos_tag(word_tokenize(x)))

In [62]:
df['ner_tweets'] = df['tweets'].apply(lambda x: ner(x))
df['ner_tweets'].head()

0                             [(what, WP), (say, VBP)]
1    [(plus, CC), (you, PRP), (ve, VBP), (add, VB),...
2    [(I, PRP), (do, VBP), (not, RB), (today, NN), ...
3    [(its, PRP$), (really, RB), (aggressive, JJ), ...
4    [(and, CC), (its, PRP$), (a, DT), (really, RB)...
Name: ner_tweets, dtype: object

### Stemming

In [63]:
# Thực hiện stemming cho các từ trong tweet sử dụng PorterStemmer
stemming = PorterStemmer()
# Áp dụng stemming cho các từ trong tweet
df['tweets'] = df['tweets'].apply(lambda x: " ".join([stemming.stem(word) for word in x.split()]))  
df['tweets'].head()

0                                             what say
1          plu you ve add commerci to the experi tacki
2    i do not today must mean i need to take anoth ...
3    it realli aggress to blast obnoxi entertain in...
4               and it a realli big bad thing about it
Name: tweets, dtype: object

### Spell Correction

In [65]:
# Kiểm tra và sửa lỗi chính tả trong văn bản
def spell_check(x):
    check = Speller(lang='en')  # Khởi tạo Speller từ thư viện autocorrect
    return check(x)  # Sửa lỗi chính tả cho văn bản

In [66]:
# Kiểm tra 10 hàng đầu tiên của DataFrame để xem dữ liệu sau khi sửa lỗi chính tả
df_sample = df.head(10).copy()  # Tạo bản sao để tránh SettingWithCopyWarning
df_sample['tweets'] = df_sample['tweets'].apply(spell_check)  # Áp dụng hàm spell_check
print(df_sample['tweets'])

0                                             what say
1            pl you ve add commerce to the expert tack
2     i do not today must mean i need to take not trip
3    it really address to blast obnoxi entertain in...
4               and it a really big bad thing about it
5    serious would pay 30 a flight for seat that do...
6    ye nearly every time i fli vx thi âear wormâ w...
7    really miss a prime opportun for man without h...
8                          well i didntâbut now i do d
9    it be ama and arxiv an hour early you re too g...
Name: tweets, dtype: object


In [67]:
# Kiểm tra chính tả sử dụng thư viện SpellChecker
import pandas as pd
from spellchecker import SpellChecker 

# Khởi tạo SpellChecker
spell = SpellChecker()


In [68]:
#Kiểm tra chính tả
def spell_check(x):
    if pd.isnull(x):  # Kiểm tra nếu x là NaN
        return x
    words = x.split()
    corrected = []
    for word in words:
        if word in spell:  # Nếu từ không sai
            corrected.append(word)
        else:
            # Lấy ứng viên từ tập hợp
            candidates = spell.candidates(word)
            if candidates:  # Kiểm tra nếu candidates không phải là None
                corrected.append(next(iter(candidates)))  # Lấy ứng viên đầu tiên
            else:
                corrected.append(word)  # Nếu không có ứng viên, thêm từ gốc
    return ' '.join(corrected)


In [69]:
def spell_check_with_progress(df):  
    # Hàm này kiểm tra và sửa lỗi chính tả cho từng dòng của cột 'tweets' trong DataFrame.
    for index, row in df.iterrows():  
        # Lặp qua từng dòng của DataFrame, trả về chỉ số (`index`) và nội dung dòng (`row`).
        df.at[index, 'tweets'] = spell_check(row['tweets'])  
        # Gọi hàm `spell_check` để kiểm tra và sửa lỗi chính tả cho nội dung trong cột 'tweets'.
        # Gán giá trị đã được sửa lỗi vào vị trí tương ứng trong DataFrame.
        if index % 100 == 0:  
            # Nếu chỉ số dòng là bội số của 100.
            print(f"Processed {index} rows")  
            # In thông báo tiến trình, cho biết đã xử lý bao nhiêu dòng.
    return df  
    # Trả về DataFrame đã được xử lý toàn bộ.

In [70]:
df.to_csv('final_df_train_second.csv')